<center>
<img src="https://raw.githubusercontent.com/FUlyankin/r_probability/master/end_seminars_2020/sem08/real_expect.png" width="500">

# Домашнее задание 6: параметрические тесты
</center>

**ФИО:**

## Общая информация

__Дата выдачи:__ 23.04.2025

__Дедлайн:__ 23:59MSK 11.05.2025

In [1]:
import numpy as np
import pandas as pd

import scipy.stats as sts
import matplotlib.pyplot as plt
import seaborn as sns
import math

### Задача №1: Лучше найди себе девушку! (25 баллов)

В этой задаче вам предстоит проверить несколько гипотез на датасете из американского сервиса знакомств. Датасет называется `dating_is_hard.csv`

Колонки датасета:
- `age` - возраст
- `is_heavy_drinker` - алкоголизм в булевом виде
- `height` - рост в дюймах
- `sex` - полов может быть только два - f для женщин и m для мужчин.
- `smokes` - зависимость от табака в булевом виде
- `location` - место жительства
- `offspring` - отношение к детям

Для каждого пункта нужно расписать нулевую и альтернативную гипотезу, описать какой именно критерий вы используете, перечислить предпослыки необходимые для его использования, а также вычислить значение статистики и p-value, если это возможно.

In [2]:
df = pd.read_csv('dating_is_hard.csv')

In [3]:
df

,Unnamed: 0,age,is_heavy_drinker,height,sex,smokes,location,offspring
0,uid_1,27,False,64,f,True,san francisco california,no opinion
1,uid_2,38,False,65,f,False,oakland california,wants kids
2,uid_3,31,False,70,m,True,san francisco california,does not have kids
3,uid_4,29,False,66,m,False,martinez california,does not have kids
4,uid_5,30,False,69,m,True,san francisco california,no opinion
...,...,...,...,...,...,...,...,...
995,uid_996,18,False,67,m,True,vallejo california,no opinion
996,uid_997,21,False,70,m,False,san francisco california,does not have kids
997,uid_998,30,False,73,m,False,oakland california,does not have kids
998,uid_999,49,False,71,m,False,san francisco california,no opinion


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        1000 non-null   object
 1   age               1000 non-null   int64 
 2   is_heavy_drinker  1000 non-null   bool  
 3   height            1000 non-null   int64 
 4   sex               1000 non-null   object
 5   smokes            1000 non-null   bool  
 6   location          1000 non-null   object
 7   offspring         1000 non-null   object
dtypes: bool(2), int64(2), object(4)
memory usage: 49.0+ KB


__a) [5 баллов]__ Рост в приложениях для знакомств не проверяют, поэтому можно поставить себе любой и никто не узнает. Посмотрим же, врут ли люди о своём росте.

Проверьте две гипотезы: совпадает ли средний рост мужчин в приложении с их реальным средним ростом и совпадает ли средний рост женщин в приложении с их реальным средним ростом

__P.S.__ Средний рост мы вам не дамим. Ищите сами и приложите ссылку на источник в решении.

In [5]:
df['height_cm'] = df['height'] * 2.54

In [6]:
men_height_cm = df[df['sex'] == 'm']['height_cm'].dropna()
women_height_cm = df[df['sex'] == 'f']['height_cm'].dropna()

In [7]:
mean_men_height = men_height_cm.mean()
mean_women_height = women_height_cm.mean()

print(f"Средний рост мужчин в приложении: {mean_men_height:.2f} см")
print(f"Средний рост женщин в приложении: {mean_women_height:.2f} см")

Средний рост мужчин в приложении: 178.57 см
Средний рост женщин в приложении: 164.67 см


In [8]:
real_men_height_cm = 175.26
real_women_height_cm = 162.56

[тык](https://worldpopulationreview.com/state-rankings/average-height-by-state#:~:text=The%20average%20height%20in%20the,tallest%20nation%20in%20the%20world.)

**Гипотезы (для мужчин):**
- H0: Средний рост мужчин в приложении равен 175.26 см
- H1: Средний рост мужчин в приложении отличается от 175.26 см

**Гипотезы (для женщин):**
- H0: Средний рост женщин в приложении равен 162.56 см
- H1: Средний рост женщин в приложении отличается от 162.56 см

**Критерий**: t-критерий

In [9]:
t_stat_men_cm, p_value_men_cm = sts.ttest_1samp(men_height_cm, real_men_height_cm)
t_stat_women_cm, p_value_women_cm = sts.ttest_1samp(women_height_cm, real_women_height_cm)

print("Мужчины:")
print(f"t-статистика = {t_stat_men_cm:.3f}, p-value = {p_value_men_cm:.3f}")
print("\nЖенщины:")
print(f"t-статистика = {t_stat_women_cm:.3f}, p-value = {p_value_women_cm:.3f}")

Мужчины:
t-статистика = 11.089, p-value = 0.000

Женщины:
t-статистика = 6.153, p-value = 0.000


__Вывод:__ люди приукрашивают свой рост в большую сторону.

__б) [10 баллов]__ По моим наблюдениям (состоящим из одного человека) мужчин-алкоголиков гораздо больше женщин. В нашем датасете алкоголизмом мы будем считать параметр `is_heavy_drinker`

Проверьте гипотезу: Действительно ли доля мужчин-алкоголиков выше доли женщин-алкоголиков?

In [10]:
n_men = df[df['sex'] == 'm'].shape[0]
n_women = df[df['sex'] == 'f'].shape[0]
x_men = df[df['sex'] == 'm']['is_heavy_drinker'].sum()
x_women = df[df['sex'] == 'f']['is_heavy_drinker'].sum()
#print(f'{n_men} {n_women} {x_men} {x_women}')

**Гипотезы:**
- H0: Доля мужчин-алкоголиков равна доле женщин-алкоголиков
- H1: Доля мужчин-алкоголиков выше, чем у женщин

**Критерий**: z-критерий

In [11]:
p1 = x_men / n_men
p2 = x_women / n_women

p_combined = (x_men + x_women) / (n_men + n_women)

se = np.sqrt(p_combined * (1 - p_combined) * (1/n_men + 1/n_women))

z_stat = (p1 - p2) / se

p_value = 1 - sts.norm.cdf(z_stat)

print(f"Доля мужчин-алкоголиков: {100*p1:.2f}%")
print(f"Доля женщин-алкоголиков: {100*p2:.2f}%")
print(f"z-статистика = {z_stat:.3f}, p-value = {p_value:.3f}")

Доля мужчин-алкоголиков: 8.50%
Доля женщин-алкоголиков: 7.75%
z-статистика = 0.423, p-value = 0.336


__Вывод:__ p-value показывает что нулевую гипотезу отвергнуть нельзя, так что пьют все!

__в) [10 баллов]__ Мой друг Володька решил бросить курить и теперь курит только когда пьёт. Это навело меня на мысль о том, что курение привлекает алкоголиков сильнее, чем обычных людей.

Проверьте гипотезу: Действительно ли алкоголики больше склонны к курению, чем неалкоголики?

**Гипотезы:**
- H0: Доля курящих одинакова среди алкоголиков и неалкоголиков
- H1: Доля курящих выше среди алкоголиков, чем среди неалкоголиков

**Критерий**: z-критерий

In [12]:
n1 = df[df['is_heavy_drinker'] == True].shape[0]
n2 = df[df['is_heavy_drinker'] == False].shape[0]

x1 = df[df['is_heavy_drinker'] == True]['smokes'].sum()
x2 = df[df['is_heavy_drinker'] == False]['smokes'].sum()

p1 = x1 / n1
p2 = x2 / n2

p_combined = (x1 + x2) / (n1 + n2)

se = np.sqrt(p_combined * (1 - p_combined) * (1/n1 + 1/n2))

z_stat = (p1 - p2) / se
p_value = 1 - sts.norm.cdf(z_stat)

print(f"Доля курящих среди алкоголиков: {100*p1:.2f}%")
print(f"Доля курящих среди неалкоголиков: {100*p2:.2f}%")
print(f"z-статистика = {z_stat:.3f}, p-value = {p_value:.3f}")

Доля курящих среди алкоголиков: 42.68%
Доля курящих среди неалкоголиков: 38.89%
z-статистика = 0.674, p-value = 0.250


**Вывод:** на основе этих данных нельзя статистически утверждать что алкоголики курят чаще

### Задача №2. Тестируем новый интерфейс страницы отеля  (25 баллов)

Все мы любим отдыхать! А злые корпорации любят наживаться на нашем желании отдыхать.

Представим что вы работаете в одной из таких корпораций и занимаетесь аналитикой сайта для бронирования отелей. Процесс бронирования имеет привычную структуру:
- вбиваете в поиск направленния и даты
- получаете выдачу вариантов отелей
- переходите на страницу отеля
- нажимаете кнопку "забронировать"
- попадаете на страницу оплаты
- оплачиваете

Очевидно, на всех этапах этой воронки вы можете потерять пользователя. Поисследовав поведение пользователей вы решили внести изменения на странице об отеле и хотите чтобы выросла конверсия в бронирования

__а) [3 балла]__ Подумайте, какие изменения могли бы быть внесены?

**Ваш ответ:** раз уж мы теряем людей на каждом этапе, возможно имеет смысл этапы как-то совместить. например уменьшить кол-во переходов и сделать так чтобы можно было оплатить и забронировать отель без перехода на его страницу, вместо этого показывать пару его лучших картинок на предыдущей странице.
еще можно создать ощущение срочности по типу "скорее забронируйте отель, пока действует скидка вместо ̶9̶9̶9̶9̶9̶9̶9̶9̶  денег заплатите всего лишь 500"

Перед вами датасет `ab_test.csv`, в нем содержатся данные о пользователях в период теста

`user_id` - уникальный идентификатор пользователя \
`sex` - пол пользователя \
`datetime` - дата и время посещения сайта (начала сессии) \
`district` - регион расположения отеля \
`hotel_page` - был ли пользователь на странице отеля в сессии \
`payment_page` - был ли пользователь на странице оплаты в сессии \
`payment` - была ли произведена оплата в сессии \
`group` - группа в эксперименте

Поскольку изменения вносились на странице отеля нас интересуют именно дошедшие до этой страницы пользователи. Поэтому колонка `hotel_page` принимает значение 1 всегда

In [13]:
df = pd.read_csv("ab_simpson.csv")
df.head()

,Unnamed: 0,user_id,sex,datetime,district,hotel_page,payment_page,payment,group
0,0,3326449108867616315,1,11/15/2023 12:02 AM,Южный федеральный округ,1,1,0,control
1,1,7887645543455909870,1,11/02/2023 09:12 PM,Центральный федеральный округ,1,1,0,control
2,2,6148146969124793824,1,11/10/2023 10:06 AM,Центральный федеральный округ,1,1,1,control
3,3,2721602951940155275,0,11/12/2023 07:10 PM,Центральный федеральный округ,1,1,0,control
4,4,1829030712785972138,0,05/01/2024 05:19 AM,Южный федеральный округ,1,0,0,test


__б) [3 балла]__ Сделайте небольшой eda, какие выводы вы можете сделать о данных? Корректно ли проводился аб тест? Найдите ошибки и объясните почему это некорректно.

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 401807 entries, 0 to 401806
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   Unnamed: 0    401807 non-null  int64 
 1   user_id       401807 non-null  int64 
 2   sex           401807 non-null  int64 
 3   datetime      401807 non-null  object
 4   district      401807 non-null  object
 5   hotel_page    401807 non-null  int64 
 6   payment_page  401807 non-null  int64 
 7   payment       401807 non-null  int64 
 8   group         401807 non-null  object
dtypes: int64(6), object(3)
memory usage: 27.6+ MB


In [15]:
print("Размер датафрейма:", df.shape)

print("\nКоличество пропусков:\n", df.isnull().sum())

print("\nУникальные значения в 'group':", df['group'].unique())

print("\nРаспределение пользователей по группам:\n", df['group'].value_counts(normalize=True))

print("\nОплаты по группам:\n", df.groupby('group')['payment'].agg(['mean', 'sum', 'count']))

print("\nРаспределение по полу в группах:\n", pd.crosstab(df['sex'], df['group'], normalize='columns'))

Размер датафрейма: (401807, 9)

Количество пропусков:
 Unnamed: 0      0
user_id         0
sex             0
datetime        0
district        0
hotel_page      0
payment_page    0
payment         0
group           0
dtype: int64

Уникальные значения в 'group': ['control' 'test']

Распределение пользователей по группам:
 group
test       0.502567
control    0.497433
Name: proportion, dtype: float64

Оплаты по группам:
              mean    sum   count
group                           
control  0.250170  50002  199872
test     0.333885  67423  201935

Распределение по полу в группах:
 group   control     test
sex                     
0      0.500275  0.49974
1      0.499725  0.50026


группы достаточно сбалансированы и по общему количеству и по составу. проблема в несбалансированности оплаты. видимо это будет нам мешать в тесте

---

пропусков в данных не обнаружено, возможно только тип даты стоило сделать правильным а так на первый взгляд все ок

Раскроем правду. Изменение продукта касалось адаптации страницы отеля под его регион. Например, в южных регионах пользователям на странице отеля предлагались активности для пляжного отдыха, а в северном и центральном регионе сразу подбирались ближайшие экскурсии и так далее.

__в) [5 баллов]__ Проанализируйте, как ведут себя конверсии в страницу оплаты и в оплату в разных регионах? Опишите выводы.

__г) [5 баллов]__ Проанализируйте, улучшилась ли воронка для пользователей в тестовой группе по сравнению с контрольной вне зависимости от региона? Опишите выводы.

__д) [5 баллов]__ Проанализируйте распределение интересов пользователей в тестовой и контрольной группе. Какими регионами они интересовались? Изменились ли их предпочтения?

Если вы все сделали правильно, то у вас получился противоречивые ответы в пунктах 3 и 4. Вы попали в ловушку Парадокса Симпсона. Подробнее о нем можно почитать [здесь](https://ru.wikipedia.org/wiki/Парадокс_Симпсона)

<img src="https://github.com/hse-econ-data-science/andan_2023/blob/main/hw/hw04/images/simpson.jpg?raw=true" width="350" align='center'>

__е) [4 балла]__ Теперь, когда вы все узнали, опишите кратко почему вы получили противоречивый результат в пунктах 3 и 4 (1 балл)

**Ваш ответ:** я в этой задаче немного запутался, пока не успел разобраться

p.s. картинка выше это буквально я сейчас, простите.

## Задача №3: Не подглядывать! (50 баллов)

<img src="https://github.com/hse-econ-data-science/andan_2023/blob/main/hw/hw04/images/trump.png?raw=true" width="500" align='left'>

На американских выборах 2020 года в некоторых штатах проходила очень конкурентная борьба между кандидатами. По мере подсчета голосов перевес в голосах склонялся то в пользу демократов, то в пользу республиканцев. И вот, в один из таких моментов, когда чаша весов склонилась в пользу республиканцев, Д. Трамп написал в твиттер "stop the count!", что означало призыв к остановке подсчета голосов.

_Здесь у вас мог возникнуть вопрос, и зачем нам это?_

Ответ очень прост. В этой ситуации всем очевидно, что призыв Трампа не мог быть осуществлен, то есть прерывать подсчет голосов раньше времени кажется довольно глупым способом выявления победителя. И вот абсолютно такой же способ принятия решения есть и в А/Б-тестах, только теперь многим аналитикам он уже кажется не столь очевидным, что приводит к существенным ошибкам в заключении выводов.

_Решив эту задачу, мы познакомимся с такой проблемой подробнее и поймем, как не стоит делать, когда вы запускаете А/Б-тест._

Чтобы понять, в чем же именно заключается проблема, мы с вами будем проводить симуляции А/Б-тестов. Это когда мы генерируем данные уже со знаниями, как эти данные устроены. А затем, запуская процедуру проверки гипотез над такими симулированными данными, мы будем оценивать, что происходит на практике (при запуске А/Б-теста).

На практике же всегда неизвестно, как устроены данные, и проводя А/Б-тест мы пытаемся зафиксировать различия. Еще раз, здесь же мы будем знать об устройстве данных, а затем будем смотреть на поведение А/Б-теста, то есть как будто бы смотреть на тест с обратной стороны.

_Перед тем как идти дальше, остановитесь здесь и еще раз зафиксируйте это!_

__а) [10 баллов]  Подсчет размера выборки для А/Б-теста__

Предположим, что вы хотите провести А/Б-тест внутри вашего приложения. Вы выбрали некоторую метрику $X$ – случайную величину из нормального распределения с мат. ожиданием $100$ у.е. и стандартным отклонением $40$ у.е – это ваш бейзлайн (группа А). Рассчитайте, сколько дней вам нужно крутить такой А/Б-тест, если:

1. Вы ожидаете получить минимальный размер эффекта в $5$ у.е.;
2. Вы хотите зафиксировать ошибку первого рода на уровне $\alpha=5\%$, а ошибку второго рода на уровне $\beta=20\%$;
3. Вы точно знаете, что в группе Б стандартное отклонение метрики составляет также $20$ у.е.
4. Посещаемость вашего приложения ежедневно составляет $200$ пользователей, и вы готовы раскатить тест на $100\%$ трафика.

Формула для расчета числа наблюдений _для одной группы_:

$$
n_{\text{one group}} = \frac{(\sigma_A^2 + \sigma_B^2) \cdot (z_{1-\frac{\alpha}{2}} + z_{1-\beta})^2}{MDE^2},
$$

__Подсказка:__ Помните, что после получения общего числа наблюдений вам нужно выразить это в дни! Вы должны получить целое число. Также будьте аккуратны с подсчетом трафика и кол-ва наблюдений – обратите внимание, со скольким числом групп вы работаете!

In [19]:
alpha = 0.05
beta = 0.20
mde = 5
std_a = 40
std_b = 40
daily_users = 200
users_per_group = daily_users // 2

z_alpha = sts.norm.ppf(1 - alpha / 2)
z_beta = sts.norm.ppf(1 - beta)

numerator = (std_a**2 + std_b**2) * (z_alpha + z_beta)**2
denominator = mde**2
n_per_group = numerator / denominator

n_per_group = math.ceil(n_per_group)

days_required = math.ceil(n_per_group / users_per_group)

print(f"Необходимое число наблюдений на группу: {n_per_group}")
print(f"Необходимое число дней теста: {days_required}")

Необходимое число наблюдений на группу: 1005
Необходимое число дней теста: 11


__б) [20 баллов] Симуляция А/Б-теста__

Теперь нам нужно запустить А/Б-тест. Для этого:

1. Сгенерируйте выборку данных из нормального распределения для группы А и Б в конкретный один день. Считайте, что в группе Б математическое ожидание составляет $105$ у.е., то есть с учетом размера вашего минимального ожидаемого эффекта из предыдущего пункта.

In [29]:
mu_a = 100
mu_b = 105
std = 40
n = 100

group_a = np.random.normal(loc=mu_a, scale=std, size=n)
group_b = np.random.normal(loc=mu_b, scale=std, size=n)

print(f"Среднее значение группы A: {np.mean(group_a):.2f}")
print(f"Среднее значение группы B: {np.mean(group_b):.2f}")

Среднее значение группы A: 93.23
Среднее значение группы B: 108.05


2. Теперь представим, что наш тест был запущен $X$ дней. Напишите функцию, которая за этот период теста собирает в `DataFrame` следующие данные на каждый день теста:
    1. среднее метрики в группе А
    2. среднее метрики в группе Б
    3. p-value, посчитанное с помощью t-test – можно воспользоваться встроенной функцией `scipy.stats.ttest_ind`
    
    Соберите `DataFrame` для $X=50$.
    
__Подсказка:__ Среднее для 1-го дня должно считаться по пользователям, пришедших в первый день, для второго дня – по пользователям, пришедшим в первый и второй день и т.д. То есть накопленным итогом за каждый день теста.

In [58]:
def simulate_ab_test(days=50, n_per_day=100, mu_a=100, mu_b=105, std=40):

    data_a = []
    data_b = []

    results = {
        'day': [],
        'mean_a': [],
        'mean_b': [],
        'p_value': []
    }

    for day in range(1, days + 1):
        day_data_a = np.random.normal(loc=mu_a, scale=std, size=n_per_day)
        day_data_b = np.random.normal(loc=mu_b, scale=std, size=n_per_day)

        data_a.extend(day_data_a)
        data_b.extend(day_data_b)

        mean_a = np.mean(data_a)
        mean_b = np.mean(data_b)
        _, p_value = sts.ttest_ind(data_a, data_b, equal_var=False)

        results['day'].append(day)
        results['mean_a'].append(mean_a)
        results['mean_b'].append(mean_b)
        results['p_value'].append(p_value)

    return pd.DataFrame(results)

ab_results = simulate_ab_test(days=50)
ab_results.head(30)

,day,mean_a,mean_b,p_value
0,1,98.908934,105.802271,0.227904
1,2,99.316918,108.741861,0.020857
2,3,101.464288,106.097919,0.157148
3,4,101.996697,107.129883,0.067491
4,5,102.686319,106.129574,0.167661
5,6,103.155977,104.381182,0.588719
6,7,102.094071,103.629569,0.463415
7,8,101.251363,104.584136,0.094616
8,9,101.741728,104.563004,0.133882
9,10,101.703452,103.950624,0.206188


3. На основе `DataFrame` из предыдущего пункта постройте график динамики среднего в группе А и в группе Б по дням теста. Отложите вертикальную линию на этом же графике, показывающую кол-во дней, посчитанных в пункте 1. Сделайте эту линию пунктирной и измените цвет, который вам больше нравится.

4. На основе `DataFrame` из предыдущего пункта постройте график динамики p-value по дням теста. Отложите вертикальную линию на этом же графике, показывающую кол-во дней, посчитанных в пункте 1. Сделайте эту линию пунктирной и измените цвет, который вам больше нравится. Также отложите горизонтальную линию – ваш уровень значимости $\alpha$.

5. Перегенируруйте шаги 2-4 несколько раз, посмотрев на графики. Что вы видите? Что происходит с p-value в периодах:
    1. до кол-ва дней, посчитанных по формуле из п.1
    2. в точке кол-ва дней, посчитанных по формуле из п.1
    3. после кол-ва дней, посчитанных по формуле из п.1
    
    Опишите выводы максимально подробно.

In [ ]:
# your thoughts here
# (◕‿◕)♡
графики у меня хорошо но вышли, но:
по первому эксперименту видно что на шаге с индексом 1 значение p-val упало ниже 0.05, а потом улетает вверх.
это может дать ложное сообщение о значимости

6. Теперь предположим, что мы поменяли наш $MDE$ до $1$ у.е. То есть мат. ожидание в группе А осталось по-прежнему $100$, а вот в группе Б теперь $101$. Перегенерируйте пункты 2-4 для измененного $MDE$, при этом кол-во дней в п.1 не пересчитывайте. Что происходит с p-value? Сделайте выводы.

In [39]:
mu_a = 100
mu_b = 101
std = 40
n = 100

np.random.seed(42)

group_a = np.random.normal(loc=mu_a, scale=std, size=n)
group_b = np.random.normal(loc=mu_b, scale=std, size=n)

print(f"Среднее значение группы A: {np.mean(group_a):.2f}")
print(f"Среднее значение группы B: {np.mean(group_b):.2f}")

ab_results = simulate_ab_test(days=50, mu_a=100, mu_b=101, std=40, n_per_day=100)
ab_results.head(30)

Среднее значение группы A: 95.85
Среднее значение группы B: 101.89


,day,mean_a,mean_b,p_value
0,1,102.595850,105.273605,0.632844
1,2,100.177836,100.830690,0.869692
2,3,100.439809,100.804986,0.911179
3,4,102.616347,101.132278,0.596643
4,5,103.374473,101.962555,0.576596
5,6,103.116434,101.569416,0.501646
6,7,104.182130,100.921518,0.127850
7,8,104.484201,101.244570,0.103605
8,9,103.767438,101.602450,0.248170
9,10,102.944276,101.461738,0.403842


__в) [20 баллов] Симуляция А/А-теста__

Теперь представим, что изменений на самом деле никаких нету, то есть мат. ожидание в обоих группах $100$, а $MDE = 0$

1. И снова перегенерируйте вышесделанные пункты 2-4. Что происходит с p-value? Может ли оно оказаться ниже уровня значимости?

In [38]:
mu_a = 100
mu_b = 100
std = 40
n = 100

group_a = np.random.normal(loc=mu_a, scale=std, size=n)
group_b = np.random.normal(loc=mu_b, scale=std, size=n)

print(f"Среднее значение группы A: {np.mean(group_a):.2f}")
print(f"Среднее значение группы B: {np.mean(group_b):.2f}")

ab_results = simulate_ab_test(days=50, mu_a=100, mu_b=100, std=40, n_per_day=100)
ab_results.head(30)

Среднее значение группы A: 98.74
Среднее значение группы B: 106.20


,day,mean_a,mean_b,p_value
0,1,102.312181,96.912910,0.383131
1,2,104.048130,97.366494,0.110409
2,3,102.547297,100.170165,0.487858
3,4,101.352646,100.409190,0.745484
4,5,101.140215,98.428803,0.293860
5,6,100.684973,98.758870,0.405134
6,7,100.821608,98.380339,0.257827
7,8,99.535737,97.805874,0.394450
8,9,99.109550,97.950403,0.546498
9,10,99.673870,97.635030,0.264072


2. Ну и напоследок, напишите функцию, которая генерирует p-value на последний день A/A теста. А затем запустите эту функцию $1000$ раз, собрав $1000$ значений p-value. Постройте гистограмму их распределения. Что вы видите?

__Подсказка:__ Для такой функции достаточно создать выборку один раз, изменив ее размер.